# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [24]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from etl import *
from queries import *
from create_tables import *

#### Creating list of filepaths to process original event csv data files

In [4]:
event_data_files = []
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
file_path = os.getcwd() + event_data_folder

# Ensure "processed_event_data_file.csv" as new processed file
processed_file_path = file_path + new_event_csv_file
if os.path.exists(processed_file_path):
    os.remove(processed_file_path)
    print('Delete file: {}'.format(new_event_csv_file))

# collect event files in @event_data_folder
print('Event data folder: ', event_data_folder)
for root, dirs, files in os.walk(file_path):
    event_data_files = glob.glob(os.path.join(root,'*'))
# print(event_data_files)

Delete file: /processed_event_data_file.csv
Event data folder:  /event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
    
# reading all event csv file 
for f in event_data_files:
    with open(f, 'r', encoding = 'utf8', newline='') as csv_file: 
        csv_reader = csv.reader(csv_file) 
        next(csv_reader)       
        for line in csv_reader:
            full_data_rows_list.append(line) 
#             print(line)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open(processed_file_path, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'first_name', 'last_name', 'gender', 'item_in_session', 'length', 'level', 'location',
                         'session_id', 'song_title', 'user_id'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
        

In [6]:
# check the number of rows in your csv file
with open(processed_file_path, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>


#### Note that event_datafile_new file named: <font color=red>processed_event_data_file.csv</font>

<img src="images/image_event_datafile_new.jpg">event data collection</img>

In [7]:
print(new_event_csv_file)

/processed_event_data_file.csv


### Prepare Cassandra instance

#### Creating a Cluster

In [25]:
# Connect to default Cluster config in local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'], port=9042)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creat Keyspace command

In [26]:
# Create a Keyspace
__default_cmd = '''
        CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
        WITH REPLICATION = {
            'class' : 'SimpleStrategy',
            'replication_factor' : 3
       }
    '''
keyspace_name = 'keyspace_test'
cmd = re.sub(r"{KEYSPACE}", keyspace_name, __default_cmd)

#### Create Keyspace & Set Keyspace to the keyspace specified above

In [27]:
try:
    session.execute(cmd)
    session.set_keyspace(keyspace_name)
    print('Keyspace "{}" created'.format(keyspace_name))
except Exception as e:
    print('Keyspace creating error: ', e)


Keyspace "keyspace_test" created


## Create tables to run the following queries

### 1. Try to clean up by dropping all related tables to avoid creating exception afterward

In [11]:
drop_tables(session)

DROP TABLE IF EXISTS songs
DROP TABLE IF EXISTS artists
DROP TABLE IF EXISTS users


### 2. Create tables

In [31]:
create_tables(session)


CREATE TABLE IF NOT EXISTS songs (
    session_id BIGINT, 
    item_in_session BIGINT,
    song_title VARCHAR,
    artist VARCHAR,
    length FLOAT, 
    PRIMARY KEY (session_id, item_in_session));

Following table created successful

CREATE TABLE IF NOT EXISTS artists (
    user_id BIGINT,
    session_id BIGINT,
    item_in_session BIGINT,
    artist VARCHAR,
    PRIMARY KEY ((user_id, session_id), item_in_session));

Following table created successful

CREATE TABLE IF NOT EXISTS users (
    song_title VARCHAR,
    user_id BIGINT, 
    first_name VARCHAR, 
    last_name VARCHAR,
    gender VARCHAR,
    location VARCHAR,
    PRIMARY KEY ((song_title), user_id));

Following table created successful


### 3. Create new csv file by processing data from event data files

In [13]:
new_file_path = process_event_data()

Delete file: /processed_event_data_file.csv
Rows: 6821
/processed_event_data_file.csv created and event data processed completed


### 4. import records into Cassandra tables

In [14]:
# insert event data into created tables
import_event_data(session, new_file_path)

Processing row 2/6821 completed
Processing row 3/6821 completed
Processing row 4/6821 completed
Processing row 5/6821 completed
Processing row 6/6821 completed
Processing row 7/6821 completed
Processing row 8/6821 completed
Processing row 9/6821 completed
Processing row 10/6821 completed
Processing row 11/6821 completed
Processing row 12/6821 completed
Processing row 13/6821 completed
Processing row 14/6821 completed
Processing row 15/6821 completed
Processing row 16/6821 completed
Processing row 17/6821 completed
Processing row 18/6821 completed
Processing row 19/6821 completed
Processing row 20/6821 completed
Processing row 21/6821 completed
Processing row 22/6821 completed
Processing row 23/6821 completed
Processing row 24/6821 completed
Processing row 25/6821 completed
Processing row 26/6821 completed
Processing row 27/6821 completed
Processing row 28/6821 completed
Processing row 29/6821 completed
Processing row 30/6821 completed
Processing row 31/6821 completed
Processing row 32/

Processing row 249/6821 completed
Processing row 250/6821 completed
Processing row 251/6821 completed
Processing row 252/6821 completed
Processing row 253/6821 completed
Processing row 254/6821 completed
Processing row 255/6821 completed
Processing row 256/6821 completed
Processing row 257/6821 completed
Processing row 258/6821 completed
Processing row 259/6821 completed
Processing row 260/6821 completed
Processing row 261/6821 completed
Processing row 262/6821 completed
Processing row 263/6821 completed
Processing row 264/6821 completed
Processing row 265/6821 completed
Processing row 266/6821 completed
Processing row 267/6821 completed
Processing row 268/6821 completed
Processing row 269/6821 completed
Processing row 270/6821 completed
Processing row 271/6821 completed
Processing row 272/6821 completed
Processing row 273/6821 completed
Processing row 274/6821 completed
Processing row 275/6821 completed
Processing row 276/6821 completed
Processing row 277/6821 completed
Processing row

Processing row 497/6821 completed
Processing row 498/6821 completed
Processing row 499/6821 completed
Processing row 500/6821 completed
Processing row 501/6821 completed
Processing row 502/6821 completed
Processing row 503/6821 completed
Processing row 504/6821 completed
Processing row 505/6821 completed
Processing row 506/6821 completed
Processing row 507/6821 completed
Processing row 508/6821 completed
Processing row 509/6821 completed
Processing row 510/6821 completed
Processing row 511/6821 completed
Processing row 512/6821 completed
Processing row 513/6821 completed
Processing row 514/6821 completed
Processing row 515/6821 completed
Processing row 516/6821 completed
Processing row 517/6821 completed
Processing row 518/6821 completed
Processing row 519/6821 completed
Processing row 520/6821 completed
Processing row 521/6821 completed
Processing row 522/6821 completed
Processing row 523/6821 completed
Processing row 524/6821 completed
Processing row 525/6821 completed
Processing row

Processing row 745/6821 completed
Processing row 746/6821 completed
Processing row 747/6821 completed
Processing row 748/6821 completed
Processing row 749/6821 completed
Processing row 750/6821 completed
Processing row 751/6821 completed
Processing row 752/6821 completed
Processing row 753/6821 completed
Processing row 754/6821 completed
Processing row 755/6821 completed
Processing row 756/6821 completed
Processing row 757/6821 completed
Processing row 758/6821 completed
Processing row 759/6821 completed
Processing row 760/6821 completed
Processing row 761/6821 completed
Processing row 762/6821 completed
Processing row 763/6821 completed
Processing row 764/6821 completed
Processing row 765/6821 completed
Processing row 766/6821 completed
Processing row 767/6821 completed
Processing row 768/6821 completed
Processing row 769/6821 completed
Processing row 770/6821 completed
Processing row 771/6821 completed
Processing row 772/6821 completed
Processing row 773/6821 completed
Processing row

Processing row 993/6821 completed
Processing row 994/6821 completed
Processing row 995/6821 completed
Processing row 996/6821 completed
Processing row 997/6821 completed
Processing row 998/6821 completed
Processing row 999/6821 completed
Processing row 1000/6821 completed
Processing row 1001/6821 completed
Processing row 1002/6821 completed
Processing row 1003/6821 completed
Processing row 1004/6821 completed
Processing row 1005/6821 completed
Processing row 1006/6821 completed
Processing row 1007/6821 completed
Processing row 1008/6821 completed
Processing row 1009/6821 completed
Processing row 1010/6821 completed
Processing row 1011/6821 completed
Processing row 1012/6821 completed
Processing row 1013/6821 completed
Processing row 1014/6821 completed
Processing row 1015/6821 completed
Processing row 1016/6821 completed
Processing row 1017/6821 completed
Processing row 1018/6821 completed
Processing row 1019/6821 completed
Processing row 1020/6821 completed
Processing row 1021/6821 co

...

Processing row 5787/6821 completed
Processing row 5788/6821 completed
Processing row 5789/6821 completed
Processing row 5790/6821 completed
Processing row 5791/6821 completed
Processing row 5792/6821 completed
Processing row 5793/6821 completed
Processing row 5794/6821 completed
Processing row 5795/6821 completed
Processing row 5796/6821 completed
Processing row 5797/6821 completed
Processing row 5798/6821 completed
Processing row 5799/6821 completed
Processing row 5800/6821 completed
Processing row 5801/6821 completed
Processing row 5802/6821 completed
Processing row 5803/6821 completed
Processing row 5804/6821 completed
Processing row 5805/6821 completed
Processing row 5806/6821 completed
Processing row 5807/6821 completed
Processing row 5808/6821 completed
Processing row 5809/6821 completed
Processing row 5810/6821 completed
Processing row 5811/6821 completed
Processing row 5812/6821 completed
Processing row 5813/6821 completed
Processing row 5814/6821 completed
Processing row 5815/

Processing row 6027/6821 completed
Processing row 6028/6821 completed
Processing row 6029/6821 completed
Processing row 6030/6821 completed
Processing row 6031/6821 completed
Processing row 6032/6821 completed
Processing row 6033/6821 completed
Processing row 6034/6821 completed
Processing row 6035/6821 completed
Processing row 6036/6821 completed
Processing row 6037/6821 completed
Processing row 6038/6821 completed
Processing row 6039/6821 completed
Processing row 6040/6821 completed
Processing row 6041/6821 completed
Processing row 6042/6821 completed
Processing row 6043/6821 completed
Processing row 6044/6821 completed
Processing row 6045/6821 completed
Processing row 6046/6821 completed
Processing row 6047/6821 completed
Processing row 6048/6821 completed
Processing row 6049/6821 completed
Processing row 6050/6821 completed
Processing row 6051/6821 completed
Processing row 6052/6821 completed
Processing row 6053/6821 completed
Processing row 6054/6821 completed
Processing row 6055/

Processing row 6267/6821 completed
Processing row 6268/6821 completed
Processing row 6269/6821 completed
Processing row 6270/6821 completed
Processing row 6271/6821 completed
Processing row 6272/6821 completed
Processing row 6273/6821 completed
Processing row 6274/6821 completed
Processing row 6275/6821 completed
Processing row 6276/6821 completed
Processing row 6277/6821 completed
Processing row 6278/6821 completed
Processing row 6279/6821 completed
Processing row 6280/6821 completed
Processing row 6281/6821 completed
Processing row 6282/6821 completed
Processing row 6283/6821 completed
Processing row 6284/6821 completed
Processing row 6285/6821 completed
Processing row 6286/6821 completed
Processing row 6287/6821 completed
Processing row 6288/6821 completed
Processing row 6289/6821 completed
Processing row 6290/6821 completed
Processing row 6291/6821 completed
Processing row 6292/6821 completed
Processing row 6293/6821 completed
Processing row 6294/6821 completed
Processing row 6295/

Processing row 6507/6821 completed
Processing row 6508/6821 completed
Processing row 6509/6821 completed
Processing row 6510/6821 completed
Processing row 6511/6821 completed
Processing row 6512/6821 completed
Processing row 6513/6821 completed
Processing row 6514/6821 completed
Processing row 6515/6821 completed
Processing row 6516/6821 completed
Processing row 6517/6821 completed
Processing row 6518/6821 completed
Processing row 6519/6821 completed
Processing row 6520/6821 completed
Processing row 6521/6821 completed
Processing row 6522/6821 completed
Processing row 6523/6821 completed
Processing row 6524/6821 completed
Processing row 6525/6821 completed
Processing row 6526/6821 completed
Processing row 6527/6821 completed
Processing row 6528/6821 completed
Processing row 6529/6821 completed
Processing row 6530/6821 completed
Processing row 6531/6821 completed
Processing row 6532/6821 completed
Processing row 6533/6821 completed
Processing row 6534/6821 completed
Processing row 6535/

Processing row 6747/6821 completed
Processing row 6748/6821 completed
Processing row 6749/6821 completed
Processing row 6750/6821 completed
Processing row 6751/6821 completed
Processing row 6752/6821 completed
Processing row 6753/6821 completed
Processing row 6754/6821 completed
Processing row 6755/6821 completed
Processing row 6756/6821 completed
Processing row 6757/6821 completed
Processing row 6758/6821 completed
Processing row 6759/6821 completed
Processing row 6760/6821 completed
Processing row 6761/6821 completed
Processing row 6762/6821 completed
Processing row 6763/6821 completed
Processing row 6764/6821 completed
Processing row 6765/6821 completed
Processing row 6766/6821 completed
Processing row 6767/6821 completed
Processing row 6768/6821 completed
Processing row 6769/6821 completed
Processing row 6770/6821 completed
Processing row 6771/6821 completed
Processing row 6772/6821 completed
Processing row 6773/6821 completed
Processing row 6774/6821 completed
Processing row 6775/

### 5. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [15]:
# Query 1:
print(select_song_query)
results = session.execute(select_song_query)
if results.current_rows:
    print(results[0])
else:
    print('No data found')



    SELECT song_title, artist, length
    FROM songs 
    WHERE session_id = 338 AND item_in_session = 4;

Row(song_title='Music Matters (Mark Knight Dub)', artist='Faithless', length=495.30731201171875)


#### 5.1 Verify that data had inserted into table "songs " correctly by browsing all records of Query 1

In [16]:
for r in results:
        print(r)

Row(song_title='Music Matters (Mark Knight Dub)', artist='Faithless', length=495.30731201171875)


### 6. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [17]:
# Query 2:
print(select_artist_query)
results = session.execute(select_artist_query)
if results.current_rows:
    print(results[0])
else:
    print('No data found')

                    


    SELECT artist, item_in_session
    FROM artists 
    WHERE user_id = 10 AND session_id = 182; 

Row(artist='Down To The Bone', item_in_session=0)


#### 6.1 Verify that data had inserted into table "artists" correctly by browsing all records of Query 2

In [18]:
# Browsing all records of Query 2
for r in results:
        print(r)

Row(artist='Down To The Bone', item_in_session=0)
Row(artist='Three Drives', item_in_session=1)
Row(artist='Sebastien Tellier', item_in_session=2)
Row(artist='Lonnie Gordon', item_in_session=3)


### 7. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [19]:
#Query 3:
print(select_user_query)
results = session.execute(select_user_query)
if results.current_rows:
    print(results[0])
else:
    print('No data found')                    


    SELECT first_name, last_name, gender, song_title, location
    FROM users
    WHERE song_title = 'All Hands Against His Own';

Row(first_name='Jacqueline', last_name='Lynch', gender='F', song_title='All Hands Against His Own', location='Atlanta-Sandy Springs-Roswell, GA')


#### 7.1 Verify that data had inserted into table "users" correctly by browsing all records of Query 3

In [20]:
for r in results:
        print(r)

Row(first_name='Jacqueline', last_name='Lynch', gender='F', song_title='All Hands Against His Own', location='Atlanta-Sandy Springs-Roswell, GA')
Row(first_name='Tegan', last_name='Levine', gender='F', song_title='All Hands Against His Own', location='Portland-South Portland, ME')
Row(first_name='Sara', last_name='Johnson', gender='F', song_title='All Hands Against His Own', location='Winston-Salem, NC')


### Drop the tables before closing out the sessions

In [21]:
drop_tables(session)

DROP TABLE IF EXISTS songs
DROP TABLE IF EXISTS artists
DROP TABLE IF EXISTS users


### Close the session and cluster connection

In [22]:
stop_session(session)

Session shutdown completed


In [23]:
stop_cluster(cluster)

Cluster shutdown completed


## View schema of Cassandra tables

In [28]:
res = session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'keyspace_test'")
print('Keyspace name: ', keyspace_name)
for r in res:
    print('Table name: {}'.format(r.table_name))

Keyspace name:  keyspace_test


In [32]:
artist_query = "SELECT * FROM system_schema.columns WHERE keyspace_name = 'keyspace_test' AND table_name = 'artists'"
res = session.execute(artist_query)
print('Table name: {}'.format(res[0].table_name))
for r in res:
    print('Column name: {}'.format(r.column_name))
    print('Column type: {}'.format(r.type))
    print('Column kind: {}'.format(r.kind))
    print('----------------------------')

Table name: artists
Column name: artist
Column type: text
Column kind: regular
----------------------------
Column name: item_in_session
Column type: bigint
Column kind: clustering
----------------------------
Column name: session_id
Column type: bigint
Column kind: partition_key
----------------------------
Column name: user_id
Column type: bigint
Column kind: partition_key
----------------------------


In [33]:
song_query = "SELECT * FROM system_schema.columns WHERE keyspace_name = 'keyspace_test' AND table_name = 'songs'"
res = session.execute(song_query)
print('Table name: {}'.format(res[0].table_name))
for r in res:
#     print(r)
    print('Column name: {}'.format(r.column_name))
    print('Column type: {}'.format(r.type))
    print('Column kind: {}'.format(r.kind))
    print('----------------------------')

Table name: songs
Column name: artist
Column type: text
Column kind: regular
----------------------------
Column name: item_in_session
Column type: bigint
Column kind: clustering
----------------------------
Column name: length
Column type: float
Column kind: regular
----------------------------
Column name: session_id
Column type: bigint
Column kind: partition_key
----------------------------
Column name: song_title
Column type: text
Column kind: regular
----------------------------


In [34]:
user_query = "SELECT * FROM system_schema.columns WHERE keyspace_name = 'keyspace_test' AND table_name = 'users'"
res = session.execute(user_query)
print('Table name: {}'.format(res[0].table_name))
for r in res:
#     print(r)
    print('Column name: {}'.format(r.column_name))
    print('Column type: {}'.format(r.type))
    print('Column kind: {}'.format(r.kind))
    print('----------------------------')

Table name: users
Column name: first_name
Column type: text
Column kind: regular
----------------------------
Column name: gender
Column type: text
Column kind: regular
----------------------------
Column name: last_name
Column type: text
Column kind: regular
----------------------------
Column name: location
Column type: text
Column kind: regular
----------------------------
Column name: song_title
Column type: text
Column kind: partition_key
----------------------------
Column name: user_id
Column type: bigint
Column kind: clustering
----------------------------
